In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# reading the excel file
df_start = pd.read_excel('worldcities.xlsx')

#get a data frame with selected columns
FORMAT = ['city', 'lat', 'lng','country','iso2','population']
df_selected = df_start[FORMAT]

#insert in the dataframe the flag for the population
df_selected["flg_pop"]=df_selected.population.apply(lambda x: 1 if x > 200000 else 0 )

# sorting values by longitude
df_selected.sort_values(by=['lng'], inplace=True, ignore_index=True)

# searching the index of London (GB)
rslt_df = df_selected.loc[(df_selected['city'] == 'London') & 
              (df_selected['iso2']=='GB')] 
index_of_london = rslt_df.index[0]

# splitting the dataframe before and after the London index
df_a = df_selected.iloc[:index_of_london]
df_b = df_selected.iloc[index_of_london:]

# reshaping of dataframe
df = df_b.append(df_a).reset_index(drop=True)

In [ ]:
class AroundTheWorld(object):
    
    def __init__(self,dataframe):
        self.dataframe=dataframe
    
    def generate_grid(self, city_start, country_start, x_size=0.2, y_size=0.1):
        start_point = self.dataframe.loc[(self.dataframe['city'] == city_start) & (self.dataframe['iso2']==country_start)][['lng','lat']].loc[0]
        point=tuple(start_point)
        y_midpoint=point #midpoint of shortest side of rectangular
        y_a=(y_midpoint[0],y_midpoint[1]-y_size/2)
        y_b=(y_midpoint[0],y_midpoint[1]+y_size/2)
        y_c=(y_b[0]+x_size,y_b[1])
        y_d=(y_c[0],y_a[1])
        print(y_midpoint, y_a, y_b, y_c, y_d)
        
    #def query(self):   

In [ ]:
around = AroundTheWorld(df)
around.generate_grid("London", "GB")

In [ ]:
plt.figure(figsize=(20,14))
plt.scatter(df.lng,df.lat, color="green", marker="o", alpha=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()